# Materials Cloud Launcher

In [ ]:
import json
from os import path
from glob import glob
import ipywidgets as ipw
from pprint import pformat
from markdown import markdown
from IPython.display import clear_output, Markdown

In [ ]:
def render():
    clear_output()
    apps = load_apps()
    config = read_config()
    for name in apps:
        accordion = mk_accordion(name)
        accordion.selected_index = None if name in config['hidden'] else 0
        display(accordion)

In [ ]:
def mk_accordion(name):
    html = ipw.HTML(load_html(name), layout=ipw.Layout(width="1000px"))
    btn_box = mk_buttons(name)
    box = ipw.HBox([html, btn_box])
    accordion = ipw.Accordion(children=[box])
    title = load_title(name)
    accordion.set_title(0, title)
    #def on_change(c):
    #    record_showhide(name, accordion.selected_index==0)
    on_change = lambda c: record_showhide(name, accordion.selected_index==0)
    accordion.observe(on_change, names="selected_index")
    return accordion

In [ ]:
def load_title(name):
    try:
        fn = "/project/apps/%s/metadata.json"%name
        metadata = json.load(open(fn))
        title = metadata['title']
    except:
        title = "%s (couldn't load title)"%s
    return title

In [ ]:
def load_apps():
    apps = [path.basename(fn) for fn in glob("/project/apps/*") if path.isdir(fn)]
    config = read_config()
    order = config['order']
    apps.sort(key=lambda x: order.index(x) if x in order else -1)
    config['order'] = apps
    write_config(config)
    return apps

In [ ]:
def load_html(name):
    try:
        fn = "/project/apps/%s/start.md"%name
        md_src = open(fn).read()
        md_src = md_src.replace("](./", "](../%s/"%name)
        html = markdown(md_src)
        
        # open links in new window/tab
        html = html.replace('<a ', '<a target="_blank" ')
        
        # downsize headings
        html = html.replace("<h3", "<h4")
        
    except:
        html = "Could not load start.md"
    return html

In [ ]:
def mk_buttons(name):
    layout = ipw.Layout(width="40px")
    btn_up   = ipw.Button(description=u"▲", layout=layout)
    btn_up.on_click(lambda b: move_updown(name, -1))
    btn_down = ipw.Button(description=u"▼", layout=layout)
    btn_down.on_click(lambda b: move_updown(name, +1))
    btn_box = ipw.VBox([btn_up, btn_down])
    return(btn_box)

In [ ]:
def move_updown(name, delta):
    config = read_config()
    order = config['order']
    n = len(order)
    i = order.index(name)
    del(order[i])
    j = min(n-1, max(0, i + delta))
    order.insert(j, name)
    config['order'] = order
    write_config(config)
    render()

In [ ]:
def record_showhide(name, visible):
    config = read_config()
    if visible:
        config['hidden'].discard(name)
    else:
        config['hidden'].add(name)
    write_config(config)

In [ ]:
CONFIG_FN = ".appmanager.conf"
def read_config():
    if path.exists(CONFIG_FN):
        return eval(open(CONFIG_FN).read())
    else:
        return {'order':[], 'hidden':set()} #default config

def write_config(config):
    with open(CONFIG_FN, "w") as f:
        f.write(pformat(config))

In [ ]:
#start the app
render()